In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 3.5 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=cae92e41db0c4e10d2dea3851a3cebbfa78385e68195ea0815e1caf3da8946ea
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 1.8 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_US         dataframe_result_csv_UK
 a0529.csv	        Github
 a0530.csv	        heatmap529_UK.ipynb
 a0531.csv	        heatmap530.ipynb
 a0601.csv	        Heat_Map_Online_language.ipynb
 csv		        __MACOSX
 csv.zip	        non_violant1.csv
 dataframe0601_UK.csv   non_violant2.csv
 dataframe_csv	        UK
 dataframe_csv_UK       violant.csv
 dataframe_result_csv  '新建 Microsoft PowerPoint 演示文稿.pptx'


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_531 = 'UK/06_02.csv'

In [8]:
ct = pd.read_csv(ct_path_531)

In [9]:
ct.shape[1]

47

In [10]:
#columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [13]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,1250734815048111_3086717874783120,Facebook,2020-06-02 23:59:55,2021-08-16 19:33:22,link,BUSTED! Democrat Rep. Engel Caught on Hot Mic ...,thegatewaypundit.com,Democrat Rep. Eliot Engel (NY) was caught on a...,Disgusting! Everything has to have a political...,1.0,https://www.thegatewaypundit.com/2020/06/buste...,https://www.facebook.com/groups/12507348150481...,28364,-4.333333,1.0,2,1,0,0,1,0,0,2,0,0,5,2,4,3,2,5,2,3,0,3,7838630,FOX NEWS with Tucker Carlson,JUTAUN,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,www.jutaun.com\n\nKings👑Brothers🛡Wizard Slayer...,False,2009-02-06 22:44:11,en,0,7838630|3086717874783120
1,2152323124789534_3102550726433431,Facebook,2020-06-02 23:59:55,2021-10-02 13:12:07,live_video_scheduled,BUSTED! Democrat Rep. Engel Caught on Hot Mic ...,thegatewaypundit.com,Democrat Rep. Eliot Engel (NY) was caught on a...,"George Floyd, Protests, Trump and more! Let's ...",1.0,https://www.facebook.com/YoungLionTV/videos/25...,https://www.facebook.com/2152323124789534/post...,461,-32.000000,2.0,0,0,0,0,0,0,0,0,0,0,2,4,6,2,0,2,0,0,0,2,11535172,Young Lion TV,YoungLionTV,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Multi racial cast of brothers giving a unique ...,False,2019-01-25 16:40:12,en,0,11535172|3102550726433431
2,3004327652955225,Facebook,2020-06-02 23:59:54,2021-09-06 11:53:10,photo,BUSTED! Democrat Rep. Engel Caught on Hot Mic ...,thegatewaypundit.com,Democrat Rep. Eliot Engel (NY) was caught on a...,George Floyd's GoFundMe has officially passed ...,2.0,https://www.facebook.com/photo.php?fbid=300432...,https://www.facebook.com/pepas.shawn/posts/300...,0,1.010667,1.0,114,129,13,90,3,0,30,0,0,12,284,4,19,44,3,16,4,1,0,5,17391053,Shawn Lettman,pepas.shawn,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_profile,US,Multi racial cast of brothers giving a unique ...,True,2019-01-25 16:40:12,en,0,17391053|3004327652955225
3,152963912041465_543126419691877,Facebook,2020-06-02 23:59:52,2021-08-15 15:25:55,photo,BUSTED! Democrat Rep. Engel Caught on Hot Mic ...,thegatewaypundit.com,"Last week, young organizers of Our Turn Twin C...",Armed police officers in schools routinely str...,2.0,https://www.facebook.com/OurTurnNatl/photos/a....,https://www.facebook.com/152963912041465/posts...,2961,1.343750,1.0,31,8,0,3,1,0,0,0,0,0,16,5,3,4,1,1,0,2,0,2,3438583,COPAL MN,copalmn,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,COPAL es una organización de base que busca im...,False,2018-03-24 15:35:01,en,0,3438583|543126419691877
4,17110490676_10157319581865677,Facebook,2020-06-02 23:59:50,2021-06-18 09:28:43,link,Beyoncé Shared Ways You Can Demand Justice for...,shape.com,,"""Justice is far from being achieved.""",1.0,https://www.shape.com/syndication/beyonce-peti...,https://www.facebook.com/17110490676/posts/101...,4215671,-1.737705,1.0,30,5,14,3,0,9,0,0,0,1,55,18,4,6,4,10,4,5,0,4,15874,SHAPE,SHAPEmagazine,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,"SHAPE Magazine offers the latest health, fitne...",True,2008-05-13 21:51:11,en,0,15874|10157319581865677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [19]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [20]:
ct['NER-msg'] = ct['message'].apply(ner)

In [21]:
ct['NER-desc'] = ct['description'].apply(ner)

In [22]:
ct['NER-title'] = ct['title'].apply(ner)

In [23]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,{},"{'NY': 1, 'Bronx': 1, 'Bronx Borough': 1}"
1,{},"{'NY': 1, 'Bronx': 1, 'Bronx Borough': 1}"
2,{},"{'NY': 1, 'Bronx': 1, 'Bronx Borough': 1}"
3,{'Minneapolis': 1},{'Minneapolis': 2}
4,{},{}


In [24]:
ct.to_csv('a0602.csv')

In [25]:
ct=pd.read_csv('a0602.csv')

In [26]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [27]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [28]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{},"{'NY': 1, 'Bronx': 1, 'Bronx Borough': 1}",{}
1,{},"{'NY': 1, 'Bronx': 1, 'Bronx Borough': 1}",{}
2,{},"{'NY': 1, 'Bronx': 1, 'Bronx Borough': 1}",{}
3,{'Minneapolis': 1},{'Minneapolis': 2},{}
5,{},"{'Asheville': 1, 'Buncombe County': 1, 'North ...",{}


In [29]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [30]:
ct_ner['NER-msg']

0                        []
1                        []
2                        []
3             [Minneapolis]
5                        []
                ...        
50765                    []
50766                    []
50767    [Allegheny County]
50769                    []
50771                    []
Name: NER-msg, Length: 36131, dtype: object

In [31]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
ct_ner['NER']

0                               [Bronx Borough, Bronx, NY]
1                               [Bronx Borough, Bronx, NY]
2                               [Bronx Borough, Bronx, NY]
3                               [Minneapolis, Minneapolis]
5        [North Carolina State, Asheville, Buncombe Cou...
                               ...                        
50765                                        [Minneapolis]
50766                                   [America, America]
50767                 [Allegheny County, America, America]
50769                                        [San Antonio]
50771                      [Minneapolis, New Jersey, N.J.]
Name: NER, Length: 36131, dtype: object

In [33]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
ner_lst = ct_ner['NER'].values

In [36]:
ner_lst

array([list(['bronx borough', 'bronx', 'ny']),
       list(['bronx borough', 'bronx', 'ny']),
       list(['bronx borough', 'bronx', 'ny']), ...,
       list(['america', 'allegheny county']), list(['san antonio']),
       list(['minneapolis', 'n.j.', 'new jersey'])], dtype=object)

In [37]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [38]:
list

['bronx borough',
 'bronx',
 'ny',
 'bronx borough',
 'bronx',
 'ny',
 'bronx borough',
 'bronx',
 'ny',
 'minneapolis',
 'north carolina state',
 'asheville',
 'buncombe county',
 'spring hill',
 'tn',
 'colorado',
 'jacksonville',
 'jacksonvile',
 'orlando',
 'minneapolis',
 'america',
 'bridgehampton',
 'minneapolis',
 'denver',
 'minneapolis',
 'minneapolis',
 'minnesota',
 'new york city',
 'chicago',
 'new york city',
 'chicago',
 'pilsen',
 'chicago',
 'pilsen',
 'chicago',
 'pilsen',
 'chicago',
 'pilsen',
 'chicago',
 'minneapolis',
 'the united states',
 'minnesota',
 'ireland',
 'paris',
 'the united states',
 'paris',
 'the united states',
 'minnesota',
 'ghana newss',
 'us',
 'pierce county',
 'tacoma',
 'australia',
 'queensland',
 'america',
 'australia',
 'america',
 'the united states',
 'australia',
 'america',
 'the united states',
 'australia',
 'america',
 'the united states',
 'houston',
 'the city of houston',
 'australia',
 'america',
 'the united states',
 'aus

In [39]:
c = collections.Counter()

In [40]:
dict_a=collections.Counter(list).most_common()

In [41]:
dict_a

[('minneapolis', 11131),
 ('us', 4488),
 ('america', 4486),
 ('minnesota', 3503),
 ('the united states', 2853),
 ('u.s.', 2399),
 ('washington', 1562),
 ('houston', 1395),
 ('atlanta', 1217),
 ('chicago', 1085),
 ('new york', 927),
 ('los angeles', 710),
 ('new york city', 639),
 ('usa', 559),
 ('d.c.', 535),
 ('texas', 533),
 ('philadelphia', 509),
 ('california', 455),
 ('boston', 445),
 ('st. louis', 434),
 ('uk', 433),
 ('las vegas', 432),
 ('nyc', 422),
 ('australia', 405),
 ('new zealand', 393),
 ('united states', 387),
 ('louisville', 386),
 ('dc', 379),
 ('seattle', 339),
 ('canada', 335),
 ('dallas', 314),
 ('china', 310),
 ('st. john’s', 301),
 ('florida', 287),
 ('georgia', 282),
 ('paris', 274),
 ('asbury park', 271),
 ('london', 244),
 ('washington dc', 242),
 ('oakland', 238),
 ('sacramento', 237),
 ('denver', 231),
 ('the united states of america', 228),
 ('north carolina', 226),
 ('riverdale', 224),
 ('nj', 222),
 ('hollywood', 220),
 ('michigan', 219),
 ('portland', 20

In [42]:
dataframe531=pd.DataFrame(dict_a)

In [43]:
len(dict_a)

5113

In [44]:
dataframe531.to_csv('dataframe0602_UK.csv')